### Import Libraries

In [38]:
try:
    from elasticsearch import Elasticsearch,helpers
    import os
    import sys
    import elasticsearch
    import pandas as pd
except Exception as e:
    print("The following error occured:",e)

In [43]:

es = Elasticsearch()

In [5]:
for filename in os.listdir():
    print(filename)

.ipynb_checkpoints
data
ElasticSearchbasics.ipynb
nlpextractor.ipynb
Untitled.ipynb


### Import File and check nulls

In [20]:
df = pd.read_csv("./data/upi/upi.csv")
print(df.isnull().sum())

UPI Banks                   0
Volume (Mn) By Costumers    0
Value (Cr) by Costumers     0
Volume (Mn)                 0
Value (Cr)                  0
Month                       0
Year                        0
dtype: int64


In [19]:
df.head()

,UPI Banks,Volume (Mn) By Costumers,Value (Cr) by Costumers,Volume (Mn),Value (Cr),Month,Year
0,Airtel Payments Bank Apps,9.04,2248.49,15.12,5732.09,12,2021
1,Airtel Payments Bank Apps,8.41,1923.95,15.90,5554.63,11,2021
2,Airtel Payments Bank Apps,7.60,1491.34,14.31,4746.79,10,2021
3,Airtel Payments Bank Apps,6.91,1520.92,11.52,3759.12,9,2021
4,Airtel Payments Bank Apps,6.14,1396.65,9.31,2941.88,8,2021


### Convert file to right format
Right format should have the following attributes
<li>Dict format</li>
<li>Unique ID</li>

In [22]:
global c 
c = 0
def ids(x):
    global c
    c = c+1
    return c
    
    

In [23]:
df["ID"] = df["Month"].apply(ids)

In [25]:
df2 = df.to_dict('records')

In [26]:
for c,line in enumerate(df2):
    print(c,line)
    break
    

0 {'UPI Banks': 'Airtel Payments Bank Apps', 'Volume (Mn) By Costumers': 9.04, 'Value (Cr) by Costumers': 2248.49, 'Volume (Mn)': 15.12, 'Value (Cr)': 5732.09, 'Month': 12, 'Year': 2021, 'ID': 1}


In [35]:
def generator(df2,database_name,table_name,id_column):
    for c,line in enumerate(df2):
        yield{
            '_index' : database_name,
            '_type' : table_name,
            '_id': line.get(id_column),
            '_source': {
                'UPI Banks':line.get('UPI Banks',['No Data']),
                'Volume (Mn) By Customers':line.get('Volume (Mn) By Costumers',['No Data']),
                'Value (Cr) by Costumers':line.get('Value (Cr) by Costumers',['No Data']),
                'Value (Cr)':line.get('Value (Cr)',['No Data']),
                'Month': line.get('Month',['No Data']),
                'Year': line.get('Year',['No Data'])
                
                # get returns the value of a key
                        
            }
        }
    raise StopIteration
    

In [36]:
gen = generator(df2,"UPI_test","UPI","ID")

In [37]:
next(gen)

{'_index': 'UPI_test',
 '_type': 'UPI',
 '_id': 1,
 '_source': {'UPI Banks': 'Airtel Payments Bank Apps',
  'Volume (Mn) By Customers': 9.04,
  'Value (Cr) by Costumers': 2248.49,
  'Value (Cr)': 5732.09,
  'Month': 12,
  'Year': 2021}}

In [51]:
es.indices.create(index="upi_test",ignore = 400)

C:\Users\parth\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'upi_test'}

In [53]:
try:
    res = helpers.bulk(es,generator(df2,"upi_test","upi","ID"))
    print("Working")
    
except Exception as e:
    print("Done",e)

Done generator raised StopIteration


In [ ]:
try:
    res = helpers.bulk(es,

In [13]:
# Database setup

In [11]:
#Lets connect to the server, we are using the python client to do so

def connect_database():
    es = None
    es = Elasticsearch([{'host':'localhost','port':9200}])
    if es.ping():
        print("connected")
    else:
        print("Failed")
    return es
connect_database()

connected


<Elasticsearch([{'host': 'localhost', 'port': 9200}])>

In [14]:
es.indices.create(index = "mydatabasename",ignore = 400)

C:\Users\parth\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'mydatabasename'}

In [17]:
res = es.indices.get_alias("*")
for name in res:
    print(name)

.kibana-event-log-7.16.3-000001
.kibana_7.16.3_001
.apm-custom-link
.kibana_task_manager_7.16.3_001
.apm-agent-configuration


C:\Users\parth\AppData\Local\Temp\ipykernel_17844\516184771.py:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.indices.get_alias("*")
C:\Users\parth\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: this request accesses system indices: [.kibana_7.16.3_001, .apm-custom-link, .kibana_task_manager_7.16.3_001, .apm-agent-configuration], but in a future major version, direct access to system indices will be prevented by default
  warnings.warn(message, category=ElasticsearchWarning)


In [16]:
es.indices.delete(index='mydatabasename',ignore = [400,404])

C:\Users\parth\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'acknowledged': True}

## Inserting data into kibana

In [19]:
record1 = {
    "name" : "Parth",
    "age" :26,
    
    
}
record2 = {
    "name" : "Samarth",
    "age" :20,
    
    
}

In [20]:
es.indices.create(index="person",ignore = 400)

C:\Users\parth\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'person'}

In [22]:
resp = es.index(index = "person",doc_type='people',body = record1)

C:\Users\parth\AppData\Local\Temp\ipykernel_17844\1122659154.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  resp = es.index(index = "person",doc_type='people',body = record1)
C:\Users\parth\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
C:\Users\parth\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: [types removal] Specifying types in document index requests is depre

In [23]:
resp

{'_index': 'person',
 '_type': 'people',
 '_id': 'VYftdX4BEJxbtSWNh2sf',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 1}

In [25]:
query = {
    "query" : {
        "match_all":{}
    }
}
    
res = es.search(index='person',body = query   )

C:\Users\parth\AppData\Local\Temp\ipykernel_17844\4077621928.py:7: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(index='person',body = query   )
C:\Users\parth\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [29]:
data = data['hits']

In [33]:
df = pd.DataFrame(data)

In [34]:
df

,_index,_type,_id,_score,_source
0,person,people,VYftdX4BEJxbtSWNh2sf,1.0,"{'name': 'Parth', 'age': 26}"
